# The SQL database for Stellab

In [345]:
import stellab_sql
reload(stellab_sql)

<module 'stellab_sql' from 'stellab_sql.py'>

Towards a stellab widget interface, similar to SYGMA and OMEGA widgets.

SQL code to create stellab database. The table structure is shown below.

Some technical aspects about our tables:
* refs: Table with literature references:
    * refid: unique reference id (PK)
    * fauthor: first author
    * year
    * nasaads: link to nasa ads page

* galaxies
    * gali: unique ref id
    * name: galaxy name

* solarnorms
    * normid: unique reference id (PK)
    * H, He... abundance entries
    * refids
* abu_table_reg
    * abu_reg_id uniqure reference
    * refid to look up literature
    * table name to look up abundance table
    * normid to look up normalization used

* multiple abundance tables, all with
    * abuid: unique reference id
    * element ratio1, element ratio2...
    * error1, errror2 ...

Rules:
* Internal connection only via (private keys) PKs.
* On user level no interaction with PKs.
* As such table entries (except PKs) can be easily modified without allowing any disconnections between tables.

Tips:
* Use not null
* fixed lenght type arrays are faster

### Add references

In [346]:
st1=st.stellab()
#papers=st1.list_ref_papers()
legs=st1.leg

name=[]
year=[]
for i,labels in enumerate(legs):
    name.append(labels.split('(')[0].strip())
    year.append(labels.split('(')[1].strip()[:-1])

In [347]:
st1.list_solar_norm()

Anders_Grevesse_1989
Grevesse_Noels_1993
Grevesse_Sauval_1998
Asplund_et_al_2009
Asplund_et_al_2005
Lodders_et_al_2009


## I. Create database with empty tables

In [348]:
stsql=stellab_sql.stellab_sql(db_name='stellab.db')

database created.


## II. Examples how to add and retrieve data.

#### Before we add abundance data we need to add a related solar normalization data and (paper) reference. The solar normalization data goes in one table (solarnorms table) and the paper reference in another (refs table).

In [349]:
#add normalization data
normalization_paper=['Anders',1989,'http://ukads.nottingham.ac.uk/abs/1993A%26A...271..587G']
normalization_label=['H','He','Fe']
normalization_data = [-2.81,0.34,0.01]
stsql.add_solar_normalization(normalization_paper,normalization_label,normalization_data)

add paper for solar normalization to table refs.
add paper related to abundance to table refs.


check that table with solar noramlization data is correctly populated:

In [350]:
#did we successfully created database entry in solar normalization table?
stsql.get_solar_normalizations()

Lets update an entry of the normalization table.

In [351]:
normalization_paper=['Anders',1989]
stsql.update_solar_normalizations(normalization_paper,update={'He':89})

found corresponding normalization data.
Solar normalization entries updated.


In [352]:
stsql.get_solar_normalizations()

In [353]:
normalization_paper=['Anders',1989]
stsql.update_solar_normalizations(normalization_paper,update={'fauthor':'Mr. X'})

found corresponding normalization data.
Column value  fauthor  belongs to paper reference table. Update value via update_paper_refs()


Check if table with paper references is correctly updated:

In [354]:
#lets see if there is an entry for the paper referene in the refs table
stsql.get_paper_refs()

In [355]:
refs_paper=['Anders',1989]
stsql.update_paper_refs(refs_paper,update={'nasads':'http://ukads.nottingham.ac.uk/abs/1993A%26A...271..587G_NEW'})

Paper reference updated.


In [356]:
stsql.get_paper_refs()

#### Add one abundance data table. A new table with abundance data is created and a new entry is added to the abu_table_reg table. The latter table holds the names and info about all abundance data tables available.

In [357]:
#add abundance data
abundance_paper=['Venn',2012,'http://adsabs.harvard.edu/abs/2012ApJ...751..102V','Milky Way']
#corresponding normalization for abundance data. Must be existant in database, else use add_solar_normalization() above.
abundance_norm=['Anders','1989']
#plain data
abundance_label=['[Fe/H]','err']
abundance_data = [-2.81,0.34]
stsql.add_abundance_data(abundance_paper,abundance_norm,abundance_label,abundance_data)

found corresponding normalization data.
add paper related to abundance to table refs.
galaxy name  Milky Way
create table  abu_table_1


In [358]:
#check if table exists, and print it.
get_abundance_paper=['Venn',2012,'Milky Way']
get_abundance_data(abu_paper=get_abundance_paper)

[(1, -1.0, -0.3)]


,abuid,Fe/H,err
0,1,-1.0,-0.3


In [359]:
#check if table exists in the registration table abu_table_reg.
#refid is the id which refers to the paper reference in the refs table which holds all references.
stsql.get_overview_abundance_tables()

In [360]:
#lets see the refs table. Indeed the refid of 2 refers to the Venn et al paper.
stsql.get_paper_refs()

In [361]:
stsql.get_galaxies()

,Galaxies
0,Milky Way


Update of table works with the same type of input as add_abundance_data()

In [362]:
abundance_data_update=[-1.0,-0.3]
stsql.update_abundance_data(abundance_paper,abundance_norm,abundance_label,abundance_data_update)

found corresponding normalization data.
abu ref paper exists already. do nothing
Abundance data table updated.


In [363]:
get_abundance_paper=['Venn',2012,'Milky Way']
stsql.get_abundance_data(abu_paper=get_abundance_paper)

[(1, -1.0, -0.3)]


,abuid,Fe/H,err
0,1,-1.0,-0.3


#### Example to add second and third abundance data tables

In [364]:
#add abundance data
stsqlabundance_paper=['BeersFun',2008,'http://adsabs.harvard.edu/abs/2012ApJ...751..102V','Milky Way']
abundance_norm=['Anders','1989']
abundance_label=['[Fe/H]','err']
abundance_data = [-99,0.8]
stsql.add_abundance_data(abundance_paper,abundance_norm,abundance_label,abundance_data)

found corresponding normalization data.
abu ref paper exists already. do nothing
abundance table with same paper reference and solar normalization exists already! Do nothing.


In [365]:
#add abundance data
abundance_paper=['BeersFun',2008,'http://adsabs.harvard.edu/abs/2012ApJ...751..102V','SMC']
abundance_norm=['Anders','1989']
abundance_label=['[Fe/H]','err']
abundance_data = [-13,0.33]
stsql.add_abundance_data(abundance_paper,abundance_norm,abundance_label,abundance_data)

found corresponding normalization data.
add paper related to abundance to table refs.
galaxy name  SMC
create table  abu_table_3


In [366]:
#as expected, the new table exists
abundance_paper=['BeersFun',2008,'Milky Way']
stsql.get_abundance_data(abu_paper=abundance_paper)

[(1, -13.0, 0.33)]


,abuid,Fe/H,err
0,1,-13.0,0.33


In [367]:
#check if table exists in abu_table_reg, the registration table. Yes, it exists as the second entry.
stsql.get_overview_abundance_tables()

In [368]:
stsql.get_galaxies()

,Galaxies
0,Milky Way
1,SMC


In [369]:
galaxy_name='Milky Way'
stsql.update_galaxy(galaxy_name,update={'Galaxies':'MW'})

Galaxy table entry updated.


In [370]:
stsql.get_galaxies()

,Galaxies
0,MW
1,SMC
